In [1]:
%autosave 60
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
from pathlib import Path

Autosaving every 60 seconds


In [2]:
p = Path("../").resolve()

In [3]:
import json
import logging
import os
import pickle
import sys
from collections import Counter
from copy import deepcopy
from io import BytesIO
from pathlib import Path
from types import ModuleType
from typing import Dict, List, Optional, Tuple, Union, cast

import cv2
import matplotlib as plt
import numpy as np
import pandas as pd
import PIL
import PIL.Image as pil_img
import seaborn as sns
import sklearn as skl
from icevision import models, tfms
from icevision.all import *
from icevision.data import Dataset, DataSplitter, RandomSplitter
from icevision.parsers.coco_parser import COCOBBoxParser
from IPython.display import Image, display
from matplotlib.patches import Rectangle
from matplotlib_inline.backend_inline import set_matplotlib_formats
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning import LightningDataModule, LightningModule, Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger
from tqdm.contrib import tenumerate, tmap, tzip
from tqdm.contrib.bells import tqdm, trange
from tqdm.notebook import tqdm

from geoscreens.geo_data import GeoScreensDataModule
from geoscreens.models import get_model, load_model_from_path
from geoscreens.modules import LightModelTorch, build_module

INFO     - The mmdet config folder already exists. No need to downloaded it. Path : /home/gbiamby/.icevision/mmdetection_configs/mmdetection_configs-2.20.1/configs | icevision.models.mmdet.download_configs:download_mmdet_configs:17


In [4]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 15)
pd.set_option("display.max_rows", 50)
# Suitable default display for floats
pd.options.display.float_format = "{:,.2f}".format
plt.rcParams["figure.figsize"] = (12, 10)

# This one is optional -- change graphs to SVG only use if you don't have a
# lot of points/lines in your graphs. Can also just use ['retina'] if you
# don't want SVG.
%config InlineBackend.figure_formats = ["retina"]
set_matplotlib_formats("pdf", "png")

In [5]:
from IPython.display import set_matplotlib_formats

set_matplotlib_formats("pdf", "png")
plt.rcParams["savefig.dpi"] = 75

plt.rcParams["figure.autolayout"] = False
plt.rcParams["figure.figsize"] = 10, 6
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["font.size"] = 16
plt.rcParams["lines.linewidth"] = 2.0
plt.rcParams["lines.markersize"] = 8
plt.rcParams["legend.fontsize"] = 14
plt.rcParams["text.usetex"] = True

plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "cm"
plt.rcParams["text.latex.preamble"] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

<ipython-input-5-55fc3ad33997>:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("pdf", "png")


In [6]:
VIDEO_PATH = Path("/shared/g-luo/geoguessr/videos").resolve()
OUT_PATH = Path("/shared/gbiamby/geo/segment").resolve()
assert VIDEO_PATH.exists()
assert OUT_PATH.exists()

files = sorted(VIDEO_PATH.glob("**/*.mp4"))
print("total video files found: ", len(files))

total video files found:  4084


In [7]:
# def sample_frames(video_path: Path, out_path: Path, sample_every_seconds=20.0):
#     """
#     Sample a frame every `sample_every_seconds` seconds from the specified video, saving it to
#     """
#     path = video_path.with_suffix(".mp4")
#     assert path.exists(), str(path)
#     cap = cv2.VideoCapture(str(path))
#     if not cap.isOpened():
#         print("could not open :", video_path)
#     num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     print(f"num_frames: {num_frames:,}")
#     seconds = round(0, 2)
#     sample_every_seconds = 5.0
#     count = 0
#     success = True
#     print(path)
#     while success:
#         cap.set(cv2.CAP_PROP_POS_MSEC, (seconds * 1000))
#         success, image = cap.read()
#         if success:
#             frame_out_path = out_path / "screen_samples_auto" / f"{path.stem}/frame_{count:08}.jpg"
#             frame_out_path.parent.mkdir(exist_ok=True, parents=True)
#             # print(frame_out_path)
#             cv2.imwrite(str(frame_out_path), image)
#         seconds = round(seconds + sample_every_seconds, 2)
#         count += 1
#     cap.release()
#     print("total frames captured: ", count, ", seconds: ", seconds)


# for i, (idx, row) in tqdm(enumerate(df_sampled.iterrows()), total=len(df_sampled)):
#     # if i > 10:
#     #     break
#     sample_frames(row.path, OUT_PATH, 20.0)

## Load Detection Model

In [8]:
seed_everything(42, workers=True)
DEVICE = torch.device("cuda:0")
config, module, model, light_model = load_model_from_path(
    "/shared/gbiamby/geo/models/best_ap_at_iou0.50/", device=DEVICE
)
geoscreens_data = GeoScreensDataModule(config, module)
light_model = light_model.eval()

Global seed set to 42


Using cfg_options:  {'model.rpn_head.anchor_generator.scales': [8], 'model.rpn_head.anchor_generator.ratios': [0.08, 0.16, 0.25, 0.36, 0.5, 0.7, 1, 2], 'model.rpn_head.anchor_generator.strides': [4, 8, 16, 32, 64]}


/home/gbiamby/anaconda3/envs/geoscreens/lib/python3.9/site-packages/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  warnings.warn(


learning_rate:  0.00055


INFO     - Loading cached records from /home/gbiamby/proj/geoscreens/datasets/geoscreens_001/dataset_cache.pkl | icevision.parsers.parser:parse:113


{'dataset_name': 'geoscreens_001', 'data_root': '/home/gbiamby/proj/geoscreens/datasets', 'img_dir': '/home/gbiamby/proj/geoscreens/datasets/images', 'img_size': 640, 'num_classes': 47, 'batch_size': 8, 'num_workers': 8, 'pin_memory': False}
classes:  <ClassMap: {'background': 0, 'between_rounds_box_white': 1, 'between_rounds_box_with_orange_next': 2, 'big_green_btw_rounds_box': 3, 'challenge_btn_orange': 4, 'challenge_high_score_board': 5, 'curr_state': 6, 'did_you_enjoy_this_location': 7, 'final_scores_box_beige': 8, 'finished_legs_box': 9, 'game_about_to_start_box_white': 10, 'game_finished_well_done_big_box': 11, 'game_finished_white_box': 12, 'game_title': 13, 'guess': 14, 'guess_grey': 15, 'guess_w_icon_only': 16, 'high_score_box': 17, 'in_game_mini_map': 18, 'invite_friends': 19, 'leader_board': 20, 'left_menu_dark': 21, 'loading_loc_white': 22, 'make_a_guess': 23, 'next_orange_btn': 24, 'next_round': 25, 'other': 26, 'participants_box': 27, 'play': 28, 'play_again': 29, 'play_c

## Run on one video

In [9]:
def image_from_url(url):
    img = PIL.Image.open(url)
    return np.array(img)


def batch(iterable, n=1):
    """Splits an iterable / list-like into batches of size n"""
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx : min(ndx + n, l)]


def segment_video(video_path: Path, output_video: bool = False):
    """
    Params:
        output_video: if True, saves output to video (untested, might not work yet)
    Returns:
        Dict: keys = frame index, value = a dict of detections that looks something like:
            {
                "label_ids": [17, 39],
                "scores": [0.5707356929779053, 0.5458141565322876],
                "bboxes": [
                    {
                        "xmin": 522.35400390625,
                        "ymin": 177.13229370117188,
                        "xmax": 640.0,
                        "ymax": 362.1326599121094,
                    },
                    {
                        "xmin": 537.4188232421875,
                        "ymin": 139.51719665527344,
                        "xmax": 635.33642578125,
                        "ymax": 157.04588317871094,
                    },
                ],
            }
    """
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("Error opening input video: {}".format(video_path))

    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"num_frames: {num_frames:,}, fps: {fps}")
    seconds = round(0, 2)
    frame_sample_rate = 4
    sample_every_seconds = 1 / float(frame_sample_rate)

    # out_ptr = None
    # if output_video:
    #     frame_width, frame_height = int(cap.get(3)), int(cap.get(4))
    #     out_ptr = cv2.VideoWriter(
    #         output_video,
    #         cv2.VideoWriter_fourcc("m", "p", "4", "v"),
    #         25,
    #         (frame_width, frame_height),
    #     )

    infer_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(640), tfms.A.Normalize()])
    frame_counter = 0
    detections = {}
    p_bar = tqdm(total=num_frames)
    while cap.isOpened():
        if frame_counter >= 200:
            break
        cap.set(cv2.CAP_PROP_POS_MSEC, (seconds * 1000))
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break

        raw_frames = [np.array(frame)]
        infer_ds = Dataset.from_images(
            raw_frames, infer_tfms, class_map=geoscreens_data.parser.class_map
        )
        batch, samples = module.build_infer_batch(infer_ds)
        preds = module.predict(model, infer_ds, detection_threshold=0.5)
        if preds:
            assert len(preds) == 1, "Expected list of size 1."
            preds = preds[0]
            detections[frame_counter] = {
                "label_ids": [int(l) for l in preds.detection.label_ids],
                "scores": preds.detection.scores.tolist(),
                "bboxes": [
                    {
                        "xmin": float(box.xmin),
                        "ymin": float(box.ymin),
                        "xmax": float(box.xmax),
                        "ymax": float(box.ymax),
                    }
                    for box in preds.detection.bboxes
                ],
            }
        # if out_ptr:
        #     # write frame into output file.
        #     out_ptr.write(new_frame)
        # else:
        #     # show the frame online, mainly used for real-time speed test.
        #     cv2.imshow("Frame", new_frame)
        #     # Press Q on keyboard to  exit
        #     if cv2.waitKey(1) & 0xFF == ord("q"):
        #         break
        frame_counter += 1
        seconds = round(seconds + sample_every_seconds, 2)
        # print(seconds, seconds * 1000)
        inc = sample_every_seconds * fps
        # print(inc)
        p_bar.update(inc)
    p_bar.close()

    print(frame_counter, seconds)
    return detections


with torch.no_grad():
    frame_detections = segment_video(VIDEO_PATH / "178LA7LXoCs.mp4")

num_frames: 45,115, fps: 29.97002997002997


  0%|          | 0/45115 [00:00<?, ?it/s]

/home/gbiamby/anaconda3/envs/geoscreens/lib/python3.9/site-packages/mmdet/core/anchor/anchor_generator.py:324: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/home/gbiamby/anaconda3/envs/geoscreens/lib/python3.9/site-packages/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  warnings.warn(


200 50.0


In [9]:
frame_detections[1]

{'label_ids': [15, 18, 41, 41, 42],
 'scores': [0.9910863041877747,
  0.9981943964958191,
  0.9994020462036133,
  0.8319348692893982,
  0.9899899959564209],
 'bboxes': [{'xmin': 525.4820556640625,
   'ymin': 484.0458068847656,
   'xmax': 631.5621337890625,
   'ymax': 495.1339416503906},
  {'xmin': 528.3643798828125,
   'ymin': 412.2805480957031,
   'xmax': 629.1768798828125,
   'ymax': 480.2262268066406},
  {'xmin': 539.5531005859375,
   'ymin': 139.9807586669922,
   'xmax': 640.0,
   'ymax': 155.9252471923828},
  {'xmin': 7.750701904296875,
   'ymin': 140.2235870361328,
   'xmax': 73.42975616455078,
   'ymax': 155.67701721191406},
  {'xmin': 484.14312744140625,
   'ymin': 160.8854217529297,
   'xmax': 627.3910522460938,
   'ymax': 179.78965759277344}]}

In [16]:
# pickle.dump(frame_detections, open(OUT_PATH / "detections/178LA7LXoCs.pkl", "wb"))

In [11]:
frame_detections = pickle.load(open(OUT_PATH / "detections/178LA7LXoCs.pkl", "rb"))

In [14]:
def get_categories():
    data = json.load(
        open("/home/gbiamby/proj/geo/datasets/geoscreens_001/geoscreens_001.json", "r")
    )
    return data["categories"]


# Note: don't use battle_royale_wait_screen for anything, it should have been labeled as backaround
cats = get_categories()
print(cats)

[{'id': 0, 'name': 'battle_royale_wait_screen'}, {'id': 1, 'name': 'between_rounds_box_white'}, {'id': 2, 'name': 'between_rounds_box_with_orange_next'}, {'id': 3, 'name': 'big_green_btw_rounds_box'}, {'id': 4, 'name': 'challenge_btn_orange'}, {'id': 5, 'name': 'challenge_high_score_board'}, {'id': 6, 'name': 'curr_state'}, {'id': 7, 'name': 'did_you_enjoy_this_location'}, {'id': 8, 'name': 'final_scores_box_beige'}, {'id': 9, 'name': 'finished_legs_box'}, {'id': 10, 'name': 'game_about_to_start_box_white'}, {'id': 11, 'name': 'game_finished_well_done_big_box'}, {'id': 12, 'name': 'game_finished_white_box'}, {'id': 13, 'name': 'game_title'}, {'id': 14, 'name': 'guess'}, {'id': 15, 'name': 'guess_grey'}, {'id': 16, 'name': 'guess_w_icon_only'}, {'id': 17, 'name': 'high_score_box'}, {'id': 18, 'name': 'in_game_mini_map'}, {'id': 19, 'name': 'invite_friends'}, {'id': 20, 'name': 'leader_board'}, {'id': 21, 'name': 'left_menu_dark'}, {'id': 22, 'name': 'loading_loc_white'}, {'id': 23, 'nam

In [26]:
# cats = geoscreens_data.parser.class_map
# print(cats)
# cats.get_by_id(0)
# cats.get_by_id(18)

<ClassMap: {'background': 0, 'between_rounds_box_white': 1, 'between_rounds_box_with_orange_next': 2, 'big_green_btw_rounds_box': 3, 'challenge_btn_orange': 4, 'challenge_high_score_board': 5, 'curr_state': 6, 'did_you_enjoy_this_location': 7, 'final_scores_box_beige': 8, 'finished_legs_box': 9, 'game_about_to_start_box_white': 10, 'game_finished_well_done_big_box': 11, 'game_finished_white_box': 12, 'game_title': 13, 'guess': 14, 'guess_grey': 15, 'guess_w_icon_only': 16, 'high_score_box': 17, 'in_game_mini_map': 18, 'invite_friends': 19, 'leader_board': 20, 'left_menu_dark': 21, 'loading_loc_white': 22, 'make_a_guess': 23, 'next_orange_btn': 24, 'next_round': 25, 'other': 26, 'participants_box': 27, 'play': 28, 'play_again': 29, 'play_current_leg': 30, 'play_next_round': 31, 'points_bar': 32, 'points_bar_two_bars': 33, 'refresh_btn': 34, 'setup_round_time_limit_box': 35, 'share_challenge_box_white': 36, 'show_full_results': 37, 'show_high_score': 38, 'start_challenge_orange': 39, 'st

'in_game_mini_map'

In [12]:
frame_detections[0]

{'label_ids': [18, 41, 15],
 'scores': [0.9932945370674133, 0.9429932832717896, 0.5963080525398254],
 'bboxes': [{'xmin': 527.5819702148438,
   'ymin': 412.2793273925781,
   'xmax': 627.6625366210938,
   'ymax': 482.1056823730469},
  {'xmin': 534.6812744140625,
   'ymin': 141.13824462890625,
   'xmax': 629.6229248046875,
   'ymax': 160.6715087890625},
  {'xmin': 526.2594604492188,
   'ymin': 483.2230529785156,
   'xmax': 625.6797485351562,
   'ymax': 494.2373962402344}]}

In [15]:
df_framedets = pd.DataFrame(
    [
        {
            "frame_id": k,
            "label_ids": v["label_ids"],
            "labels": [cats[l]["name"] for l in v["label_ids"]],
            "labels_set": tuple(set(cats[l]["name"] for l in v["label_ids"])),
            "scores": v["scores"],
            "bboxes": v["bboxes"],
        }
        for k, v in frame_detections.items()
    ]
)
df_framedets

,frame_id,label_ids,labels,labels_set,scores,bboxes
0,0,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9932945370674133, 0.9429932832717896, 0.5963080525398254]","[{'xmin': 527.5819702148438, 'ymin': 412.2793273925781, 'xmax': 627.6625366210938, 'ymax': 482.1056823730469}, {'xmin': 534.6812744140625, 'ymin': 141.13824462890625, 'xmax': 629.6229248046875, 'ymax': 160.6715087890625}, {'xmin': 526.2594604492188, 'ymin': 483.2230529785156, 'xmax': 625.6797485351562, 'ymax': 494.2373962402344}]"
1,1,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.993131697177887, 0.9442188143730164, 0.5944535732269287]","[{'xmin': 527.5426025390625, 'ymin': 412.3382873535156, 'xmax': 627.6483154296875, 'ymax': 482.1278381347656}, {'xmin': 534.7284545898438, 'ymin': 141.13693237304688, 'xmax': 629.7432250976562, 'ymax': 160.68435668945312}, {'xmin': 526.2078247070312, 'ymin': 483.2252197265625, 'xmax': 625.5960083007812, 'ymax': 494.23895263671875}]"
2,2,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9931173324584961, 0.9441524147987366, 0.5962186455726624]","[{'xmin': 527.5955200195312, 'ymin': 412.3570556640625, 'xmax': 627.6630249023438, 'ymax': 482.18011474609375}, {'xmin': 534.7642211914062, 'ymin': 141.1367645263672, 'xmax': 629.7544555664062, 'ymax': 160.68309020996094}, {'xmin': 526.2374877929688, 'ymin': 483.23785400390625, 'xmax': 625.6256713867188, 'ymax': 494.24810791015625}]"
3,3,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9931808710098267, 0.9441347718238831, 0.5958788990974426]","[{'xmin': 527.6317138671875, 'ymin': 412.35272216796875, 'xmax': 627.663818359375, 'ymax': 482.16839599609375}, {'xmin': 534.8040771484375, 'ymin': 141.12327575683594, 'xmax': 629.7984619140625, 'ymax': 160.6488494873047}, {'xmin': 526.3035888671875, 'ymin': 483.2323303222656, 'xmax': 625.664794921875, 'ymax': 494.2460632324219}]"
4,4,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9934619665145874, 0.9446057081222534, 0.5995933413505554]","[{'xmin': 527.5458984375, 'ymin': 412.35247802734375, 'xmax': 627.5087890625, 'ymax': 482.0555419921875}, {'xmin': 534.8361206054688, 'ymin': 141.12213134765625, 'xmax': 629.9126586914062, 'ymax': 160.62973022460938}, {'xmin': 526.291748046875, 'ymin': 483.2200012207031, 'xmax': 625.7034912109375, 'ymax': 494.2319641113281}]"
...,...,...,...,...,...,...
6017,6017,"[5, 21, 41, 17]","[challenge_high_score_board, left_menu_dark, status_bar, high_score_box]","(left_menu_dark, challenge_high_score_board, status_bar, high_score_box)","[0.9724621176719666, 0.9249890446662903, 0.5198752880096436, 0.5176929235458374]","[{'xmin': 83.38980102539062, 'ymin': 366.0386962890625, 'xmax': 564.558837890625, 'ymax': 493.57171630859375}, {'xmin': 1.4149627685546875, 'ymin': 189.72463989257812, 'xmax': 93.16571044921875, 'ymax': 501.0010070800781}, {'xmin': 0.0, 'ymin': 141.4392547607422, 'xmax': 144.798095703125, 'ymax': 157.8351593017578}, {'xmin': 470.8642883300781, 'ymin': 345.10882568359375, 'xmax': 611.505615234375, 'ymax': 501.46478271484375}]"
6018,6018,"[5, 21, 41, 17]","[challenge_high_score_board, left_menu_dark, status_bar, high_score_box]","(left_menu_dark, challenge_high_score_board, status_bar, high_score_box)","[0.9725633859634399, 0.9207849502563477, 0.5252230167388916, 0.5169333815574646]","[{'xmin': 83.35728454589844, 'ymin': 365.9983825683594, 'xmax': 564.5224609375, 'ymax': 493.5742492675781}, {'xmin': 1.4939422607421875, 'ymin': 188.9840087890625, 'xmax': 93.10298919677734, 'ymax': 500.25408935546875}, {'xmin': 0.0, 'ymin': 141.39881896972656, 'xmax': 144.77581787109375, 'ymax': 157.81724548339844}, {'xmin': 470.8016357421875, 'ymin': 345.1438903808594, 'xmax': 611.49755859375, 'ymax': 501.4822082519531}]"
6019,6019

In [16]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(pd.DataFrame(df_framedets.labels_set.value_counts()))

,labels_set
"(status_bar, status_bar_white, in_game_mini_map)",1148
"(guess_grey, status_bar, in_game_mini_map)",814
"(status_bar, game_title, status_bar_white, in_game_mini_map)",348
"(status_bar, in_game_mini_map)",320
"(guess_grey, status_bar, status_bar_white, in_game_mini_map)",314
"(points_bar_two_bars, between_rounds_box_white, status_bar)",296
"(left_menu_dark, challenge_high_score_board, status_bar)",275
(),268
"(left_menu_dark, challenge_high_score_board)",244
"(left_menu_dark, status_bar, game_title, challenge_high_score_board)",241


In [17]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(pd.DataFrame(df_framedets.labels_set.value_counts()))
df_framedets["ingame_trio"] = df_framedets.labels_set.apply(
    lambda x: "in_game_mini_map" in x and "status_bar" in x and "guess_grey" in x
)
df_framedets

,frame_id,label_ids,labels,labels_set,scores,bboxes,ingame_trio
0,0,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9932945370674133, 0.9429932832717896, 0.5963080525398254]","[{'xmin': 527.5819702148438, 'ymin': 412.2793273925781, 'xmax': 627.6625366210938, 'ymax': 482.1056823730469}, {'xmin': 534.6812744140625, 'ymin': 141.13824462890625, 'xmax': 629.6229248046875, 'ymax': 160.6715087890625}, {'xmin': 526.2594604492188, 'ymin': 483.2230529785156, 'xmax': 625.6797485351562, 'ymax': 494.2373962402344}]",True
1,1,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.993131697177887, 0.9442188143730164, 0.5944535732269287]","[{'xmin': 527.5426025390625, 'ymin': 412.3382873535156, 'xmax': 627.6483154296875, 'ymax': 482.1278381347656}, {'xmin': 534.7284545898438, 'ymin': 141.13693237304688, 'xmax': 629.7432250976562, 'ymax': 160.68435668945312}, {'xmin': 526.2078247070312, 'ymin': 483.2252197265625, 'xmax': 625.5960083007812, 'ymax': 494.23895263671875}]",True
2,2,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9931173324584961, 0.9441524147987366, 0.5962186455726624]","[{'xmin': 527.5955200195312, 'ymin': 412.3570556640625, 'xmax': 627.6630249023438, 'ymax': 482.18011474609375}, {'xmin': 534.7642211914062, 'ymin': 141.1367645263672, 'xmax': 629.7544555664062, 'ymax': 160.68309020996094}, {'xmin': 526.2374877929688, 'ymin': 483.23785400390625, 'xmax': 625.6256713867188, 'ymax': 494.24810791015625}]",True
3,3,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9931808710098267, 0.9441347718238831, 0.5958788990974426]","[{'xmin': 527.6317138671875, 'ymin': 412.35272216796875, 'xmax': 627.663818359375, 'ymax': 482.16839599609375}, {'xmin': 534.8040771484375, 'ymin': 141.12327575683594, 'xmax': 629.7984619140625, 'ymax': 160.6488494873047}, {'xmin': 526.3035888671875, 'ymin': 483.2323303222656, 'xmax': 625.664794921875, 'ymax': 494.2460632324219}]",True
4,4,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9934619665145874, 0.9446057081222534, 0.5995933413505554]","[{'xmin': 527.5458984375, 'ymin': 412.35247802734375, 'xmax': 627.5087890625, 'ymax': 482.0555419921875}, {'xmin': 534.8361206054688, 'ymin': 141.12213134765625, 'xmax': 629.9126586914062, 'ymax': 160.62973022460938}, {'xmin': 526.291748046875, 'ymin': 483.2200012207031, 'xmax': 625.7034912109375, 'ymax': 494.2319641113281}]",True
...,...,...,...,...,...,...,...
6017,6017,"[5, 21, 41, 17]","[challenge_high_score_board, left_menu_dark, status_bar, high_score_box]","(left_menu_dark, challenge_high_score_board, status_bar, high_score_box)","[0.9724621176719666, 0.9249890446662903, 0.5198752880096436, 0.5176929235458374]","[{'xmin': 83.38980102539062, 'ymin': 366.0386962890625, 'xmax': 564.558837890625, 'ymax': 493.57171630859375}, {'xmin': 1.4149627685546875, 'ymin': 189.72463989257812, 'xmax': 93.16571044921875, 'ymax': 501.0010070800781}, {'xmin': 0.0, 'ymin': 141.4392547607422, 'xmax': 144.798095703125, 'ymax': 157.8351593017578}, {'xmin': 470.8642883300781, 'ymin': 345.10882568359375, 'xmax': 611.505615234375, 'ymax': 501.46478271484375}]",False
6018,6018,"[5, 21, 41, 17]","[challenge_high_score_board, left_menu_dark, status_bar, high_score_box]","(left_menu_dark, challenge_high_score_board, status_bar, high_score_box)","[0.9725633859634399, 0.9207849502563477, 0.5252230167388916, 0.5169333815574646]","[{'xmin': 83.35728454589844, 'ymin': 365.9983825683594, 'xmax': 564.5224609375, 'ymax': 493.5742492675781}, {'xmin': 1.4939422607421875, 'ymin': 188.9840087890625, 'xmax': 93.10298919677734, 'ymax': 500.25408935546875}, {'xmin': 0.0, 'ymin': 141.39881896972656, 'xmax': 144.77581787109375, 'ymax': 157.81724548339844}, {'xmin': 470.8016357421875, 'ymin': 345.1438903808594, 'xmax': 611.4975

In [18]:
pd.DataFrame(df_framedets.groupby(["labels_set"]).agg(cnt=("frame_id", "count")))

,cnt
labels_set,
(),268
"(between_rounds_box_white, did_you_enjoy_this_location, status_bar)",26
"(between_rounds_box_white, did_you_enjoy_this_location, status_bar, game_title)",1
"(between_rounds_box_white, did_you_enjoy_this_location, status_bar, points_bar_two_bars)",8
"(between_rounds_box_white, points_bar_two_bars, status_bar)",7
"(between_rounds_box_white, points_bar_two_bars, status_bar, did_you_enjoy_this_location)",11
"(between_rounds_box_white, status_bar)",5
"(challenge_high_score_board,)",1
"(game_title, status_bar, status_bar_white, guess_grey, in_game_mini_map)",182


In [19]:
df_framedets["label_set_count"] = df_framedets.merge(
    pd.DataFrame(df_framedets.groupby(["labels_set"]).agg(cnt=("frame_id", "count"))),
    left_on="labels_set",
    right_on="labels_set",
)["cnt"]

In [31]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
# display(df_framedets[~df_framedets.ingame_trio].sort_values(["label_set_count"], ascending=False))

In [20]:
df_framedets

,frame_id,label_ids,labels,labels_set,scores,bboxes,ingame_trio,label_set_count
0,0,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9932945370674133, 0.9429932832717896, 0.5963080525398254]","[{'xmin': 527.5819702148438, 'ymin': 412.2793273925781, 'xmax': 627.6625366210938, 'ymax': 482.1056823730469}, {'xmin': 534.6812744140625, 'ymin': 141.13824462890625, 'xmax': 629.6229248046875, 'ymax': 160.6715087890625}, {'xmin': 526.2594604492188, 'ymin': 483.2230529785156, 'xmax': 625.6797485351562, 'ymax': 494.2373962402344}]",True,814
1,1,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.993131697177887, 0.9442188143730164, 0.5944535732269287]","[{'xmin': 527.5426025390625, 'ymin': 412.3382873535156, 'xmax': 627.6483154296875, 'ymax': 482.1278381347656}, {'xmin': 534.7284545898438, 'ymin': 141.13693237304688, 'xmax': 629.7432250976562, 'ymax': 160.68435668945312}, {'xmin': 526.2078247070312, 'ymin': 483.2252197265625, 'xmax': 625.5960083007812, 'ymax': 494.23895263671875}]",True,814
2,2,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9931173324584961, 0.9441524147987366, 0.5962186455726624]","[{'xmin': 527.5955200195312, 'ymin': 412.3570556640625, 'xmax': 627.6630249023438, 'ymax': 482.18011474609375}, {'xmin': 534.7642211914062, 'ymin': 141.1367645263672, 'xmax': 629.7544555664062, 'ymax': 160.68309020996094}, {'xmin': 526.2374877929688, 'ymin': 483.23785400390625, 'xmax': 625.6256713867188, 'ymax': 494.24810791015625}]",True,814
3,3,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9931808710098267, 0.9441347718238831, 0.5958788990974426]","[{'xmin': 527.6317138671875, 'ymin': 412.35272216796875, 'xmax': 627.663818359375, 'ymax': 482.16839599609375}, {'xmin': 534.8040771484375, 'ymin': 141.12327575683594, 'xmax': 629.7984619140625, 'ymax': 160.6488494873047}, {'xmin': 526.3035888671875, 'ymin': 483.2323303222656, 'xmax': 625.664794921875, 'ymax': 494.2460632324219}]",True,814
4,4,"[18, 41, 15]","[in_game_mini_map, status_bar, guess_grey]","(guess_grey, status_bar, in_game_mini_map)","[0.9934619665145874, 0.9446057081222534, 0.5995933413505554]","[{'xmin': 527.5458984375, 'ymin': 412.35247802734375, 'xmax': 627.5087890625, 'ymax': 482.0555419921875}, {'xmin': 534.8361206054688, 'ymin': 141.12213134765625, 'xmax': 629.9126586914062, 'ymax': 160.62973022460938}, {'xmin': 526.291748046875, 'ymin': 483.2200012207031, 'xmax': 625.7034912109375, 'ymax': 494.2319641113281}]",True,814
...,...,...,...,...,...,...,...,...
6017,6017,"[5, 21, 41, 17]","[challenge_high_score_board, left_menu_dark, status_bar, high_score_box]","(left_menu_dark, challenge_high_score_board, status_bar, high_score_box)","[0.9724621176719666, 0.9249890446662903, 0.5198752880096436, 0.5176929235458374]","[{'xmin': 83.38980102539062, 'ymin': 366.0386962890625, 'xmax': 564.558837890625, 'ymax': 493.57171630859375}, {'xmin': 1.4149627685546875, 'ymin': 189.72463989257812, 'xmax': 93.16571044921875, 'ymax': 501.0010070800781}, {'xmin': 0.0, 'ymin': 141.4392547607422, 'xmax': 144.798095703125, 'ymax': 157.8351593017578}, {'xmin': 470.8642883300781, 'ymin': 345.10882568359375, 'xmax': 611.505615234375, 'ymax': 501.46478271484375}]",False,3
6018,6018,"[5, 21, 41, 17]","[challenge_high_score_board, left_menu_dark, status_bar, high_score_box]","(left_menu_dark, challenge_high_score_board, status_bar, high_score_box)","[0.9725633859634399, 0.9207849502563477, 0.5252230167388916, 0.5169333815574646]","[{'xmin': 83.35728454589844, 'ymin': 365.9983825683594, 'xmax': 564.5224609375, 'ymax': 493.5742492675781}, {'xmin': 1.4939422607421875, 'ymin': 188.9840087890625, 'xmax': 93.10298919677734, 'ymax': 500.25408935546875}, {'xmin': 0.0, 'ymin': 141.39881896972656, 'xmax': 144.77581787109375, 'ymax': 157.81724548339844}, {'xmin': 470.8016357421875, '

In [21]:
cats

[{'id': 0, 'name': 'battle_royale_wait_screen'},
 {'id': 1, 'name': 'between_rounds_box_white'},
 {'id': 2, 'name': 'between_rounds_box_with_orange_next'},
 {'id': 3, 'name': 'big_green_btw_rounds_box'},
 {'id': 4, 'name': 'challenge_btn_orange'},
 {'id': 5, 'name': 'challenge_high_score_board'},
 {'id': 6, 'name': 'curr_state'},
 {'id': 7, 'name': 'did_you_enjoy_this_location'},
 {'id': 8, 'name': 'final_scores_box_beige'},
 {'id': 9, 'name': 'finished_legs_box'},
 {'id': 10, 'name': 'game_about_to_start_box_white'},
 {'id': 11, 'name': 'game_finished_well_done_big_box'},
 {'id': 12, 'name': 'game_finished_white_box'},
 {'id': 13, 'name': 'game_title'},
 {'id': 14, 'name': 'guess'},
 {'id': 15, 'name': 'guess_grey'},
 {'id': 16, 'name': 'guess_w_icon_only'},
 {'id': 17, 'name': 'high_score_box'},
 {'id': 18, 'name': 'in_game_mini_map'},
 {'id': 19, 'name': 'invite_friends'},
 {'id': 20, 'name': 'leader_board'},
 {'id': 21, 'name': 'left_menu_dark'},
 {'id': 22, 'name': 'loading_loc_wh

In [79]:
def classify_frame(dets: dict):
    label_set = set(dets["labels_set"])

    if label_set in [
        set(["points_bar_two_bars", "did_you_enjoy_this_location", "status_bar"]),
        set(["between_rounds_box_white", "did_you_enjoy_this_location", "status_bar"]),
        set(["points_bar_two_bars", "status_bar"]),
        set(["points_bar_two_bars", "status_bar", "game_title"]),
        set(["points_bar_two_bars", "did_you_enjoy_this_location", "status_bar", "game_title"]),
        set(["points_bar_two_bars", "status_bar", "share_challenge_box_white"]),
        set(["points_bar_two_bars", "challenge_high_score_board", "status_bar"]),
        set(["points_bar_two_bars", "status_bar", "game_title", "share_challenge_box_white"]),
        set(
            ["between_rounds_box_white", "did_you_enjoy_this_location", "status_bar", "game_title"]
        ),
        # set([]),
    ]:
        return "between_round_or_game_ambiguous"
    # elif label_set in [
    #     set(["points_bar_two_bars", "did_you_enjoy_this_location", "status_bar"]),
    # ]:
    #     return "between_round_or_game_ambiguous"
    elif "in_game_mini_map" in label_set and "status_bar" in label_set:
        return "in_game"
    elif "left_menu_dark" in label_set and "challenge_high_score_board" in label_set:
        return "between_games"
    elif "points_bar_two_bars" in label_set and "between_rounds_box_white" in label_set:
        return "between_rounds"
    elif "left_menu_dark" in label_set:
        return "between_games"
    elif "points_bar_two_bars" in label_set:
        return "unknown"
    elif len(label_set) == 0:
        return "out_of_game"
    else:
        return "unknown"


df_framedets["game_state"] = df_framedets.apply(classify_frame, axis=1)

# ui_to_status = {
#     tuple(): "out-of-game",
#     tuple(sorted(["status_bar", "in_game_mini_map", "guess_grey"])): "in-game",
# }
# for i, (frame_id, frame_dets) in enumerate(frame_detections.items()):
#     if i >= 120:
#         break
#     print("")
#     print(f"{i/4.0:04} sec")
#     labels = tuple(sorted([cats[l]["name"] for l in frame_dets["label_ids"]]))
#     print(labels)
#     # for in ui_to_status.items()
#     print(classify_frame(frame_dets))

In [80]:
pd.DataFrame(df_framedets.game_state.value_counts())

,game_state
in_game,3598
between_games,1167
between_rounds,544
between_round_or_game_ambiguous,338
out_of_game,268
unknown,107


In [81]:
pd.DataFrame(
    df_framedets[df_framedets.game_state == "unknown"]
    .groupby(["labels_set"])
    .agg(cnt=("frame_id", "count"))
).sort_values("cnt", ascending=False)

,cnt
labels_set,
"(status_bar,)",39
"(status_bar, game_title)",31
"(in_game_mini_map,)",16
"(status_bar, status_bar_white)",11
"(between_rounds_box_white, status_bar)",5
"(status_bar, game_title, share_challenge_box_white)",2
"(status_bar, game_title, status_bar_white)",2
"(challenge_high_score_board,)",1


In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_framedets.game_state)

In [ ]:
# def classify_frame(dets: dict):
#     return [
#         (label_id, cats[label_id], score, "")
#         for label_id, score, bboxes in zip(dets["label_ids"], dets["scores"], dets["bboxes"])
#     ]


# ui_to_status = {
#     tuple(): "out-of-game",
#     tuple(sorted(['status_bar', 'in_game_mini_map', 'guess_grey'])): "in-game",

# }
# for i, (frame_id, frame_dets) in enumerate(frame_detections.items()):
#     if i >= 120:
#         break
#     print("")
#     print(f"{i/4.0:04} sec")
#     frame_labels = tuple(sorted([cats[l]["name"] for l in frame_dets["label_ids"]]))
#     print(frame_labels)
#     for _labels, status in ui_to_status.items():
#         for l in _labels:
#             if l in frame_labels
#     if labels in ui_to_status:
#         print("status: ", ui_to_status[labels])
#     print(classify_frame(frame_dets))

In [62]:
p = Path("./").resolve()
p

Path('/home/gbiamby/proj/geo/notebooks')

In [64]:
p.name

'notebooks'

### Scratch: Fix the cscv's that didn't get header rows:

In [35]:
# files = sorted(Path("/shared/gbiamby/geo/segment/detections/train").glob("*.csv"))

# cols = [c for c in df_framedets.columns]
# cols = ["index"] + cols
# print("cols: ", cols)
# for f in files:
#     # print("1")
#     df = pd.read_csv(f)
#     # print(df.shape, df.columns)
#     df.columns = cols
#     df = df.drop(columns=["index"])
#     df.to_csv(f, header=True, index=False)
# print(df_framedets.columns)
# print(df.columns)
# df

cols:  ['index', 'frame_id', 'label_ids', 'labels', 'labels_set', 'scores', 'bboxes', 'ingame_trio', 'label_set_count']
Index(['frame_id', 'label_ids', 'labels', 'labels_set', 'scores', 'bboxes',
       'ingame_trio', 'label_set_count'],
      dtype='object')
Index(['frame_id', 'label_ids', 'labels', 'labels_set', 'scores', 'bboxes',
       'ingame_trio', 'label_set_count'],
      dtype='object')


,frame_id,label_ids,labels,labels_set,scores,bboxes,ingame_trio,label_set_count
0,0,[],[],(),[],[],False,93
1,1,[],[],(),[],[],False,93
2,2,[],[],(),[],[],False,93
3,3,[],[],(),[],[],False,93
4,4,[],[],(),[],[],False,93
...,...,...,...,...,...,...,...,...
4480,4480,[],[],(),[],[],False,17
4481,4481,[],[],(),[],[],False,17
4482,4482,[],[],(),[],[],False,17
4483,4483,[],[],(),[],[],False,17


In [38]:
df.describe()

,frame_id,label_set_count
count,"4,485.00","4,485.00"
mean,"2,242.00","1,380.94"
std,"1,294.85",916.99
min,0.00,1.00
25%,"1,121.00",304.00
50%,"2,242.00","2,249.00"
75%,"3,363.00","2,249.00"
max,"4,484.00","2,249.00"


LightModelMMDet(
  (model): FasterRCNN(
    (backbone): ResNeSt(
      (stem): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): SyncBatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): SyncBatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (7): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): ReLU(inplace=True)
      )
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): ResLayer(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): SyncBat